In [45]:
import numpy as np
import os
import pandas as pd
import ast

import json
import re
import yaml

In [80]:

def format_columns(df, columns):
    """
    Formats specified columns of a pandas dataframe with a precision of 2 decimal places
    and exponent notation.
    
    Parameters:
    df (pandas.DataFrame): The dataframe to format.
    columns (list of str): A list of column names to format.
    
    Returns:
    pandas.DataFrame: The formatted dataframe.
    """
    df_formatted = df.copy()  # create a copy of the dataframe to modify
    df_formatted[columns] = df[columns].applymap(lambda x: f'{x:.2e}'.replace('e-', 'x10^-'))  # apply formatting to specified columns
    return df_formatted

In [46]:
t7_drive = '/media/ak/T7/August11th2022Experiments/'
infoClockFolders = 'InfoClockDataFrames'
mmdOutputFiles = 'LinearMMDOutputFiles'
mmdOutputFilesLoc = os.path.join(t7_drive, mmdOutputFiles)
mmdOutputLoc = os.path.join(t7_drive, mmdOutputFiles)
tables = '/media/ak/T7/August11th2022Experiments/TablesOfResults'
os.listdir(mmdOutputFilesLoc)

['G_1_tick_processedLinearMMDresults_tau.pickle',
 'G_1_volume_processedLinearMMDresults_tau.pickle',
 'G_1_dollar_processedLinearMMDresults_tau.pickle',
 'G_1_dollar_processedLinearMMDresults_alpha.pickle',
 'XM1_dollar_processedLinearMMDresults_alpha.pickle',
 'G_1_calendar_processedLinearMMDresults_alpha.pickle',
 'G_1_volume_processedLinearMMDresults_alpha.pickle',
 'G_1_tick_processedLinearMMDresults_alpha.pickle',
 'XM1_volume_processedLinearMMDresults_tau.pickle',
 'XM1_dollar_processedLinearMMDresults_tau.pickle',
 'TY1_dollar_processedLinearMMDresults_tau.pickle',
 'G_1_calendar_processedLinearMMDresults_tau.pickle',
 'TY1_tick_processedLinearMMDresults_tau.pickle',
 'TY1_volume_processedLinearMMDresults_tau.pickle',
 'TY1_volume_processedLinearMMDresults_alpha.pickle',
 'TY1_tick_processedLinearMMDresults_alpha.pickle',
 'TY1_calendar_processedLinearMMDresults_alpha.pickle',
 'TY1_calendar_processedLinearMMDresults_tau.pickle',
 'TY1_dollar_processedLinearMMDresults_alpha.pic

In [47]:
def parse_string_to_dict(s):
    s = re.sub(r"(\w+):", r'"\1":', s)  # Add quotes around keys
    s = s.replace("'", '"')  # Replace single quotes with double quotes
    return json.loads(s)  # Parse the resulting JSON string

In [48]:


def split_nan(df):
    """
    Splits a Pandas DataFrame into two separate DataFrames,
    one containing the NaN values and the other containing
    the non-NaN values.
    """
    # create a boolean mask that flags all NaN values as True
    mask = df.isna()

    # split the DataFrame into two separate DataFrames using boolean indexing
    nan_df = df[mask]
    non_nan_df = df[~mask]

    return nan_df, non_nan_df


In [49]:
class ProcessFiles:

    def __init__(self, symbol, mmd_output_files_loc, file_types=None):
        self.symbol = symbol
        self.mmd_output_files_loc = mmd_output_files_loc
        if file_types is None:
            self.file_types = ['volume', 'tick', 'calendar', 'dollar']
        else:
            self.file_types = file_types
    
    def generate_file_paths(self):
        file_paths = {}
        for file_type in self.file_types:
            file_path = os.path.join(self.mmd_output_files_loc, f'{self.symbol}_{file_type}_processedLinearMMDresults_alpha.pickle')
            if os.path.exists(file_path):
                file_paths[file_type] = file_path
        return file_paths
    
    def read_pickles(self):
        file_paths = self.generate_file_paths()
        dataframes = {}
        for file_type, file_path in file_paths.items():
            if os.path.exists(file_path):
                loaded_data = pd.read_pickle(file_path)
                if isinstance(loaded_data, dict):
                    loaded_data = pd.DataFrame(loaded_data)
                dataframes[file_type] = loaded_data
        return dataframes

    def extract_data(self, df):
        df['alpha_1'] = df['Test Results 1'].apply(lambda x: x['test_result']['alpha'])
        df['pvalue_1'] = df['Test Results 1'].apply(lambda x: x['test_result']['pvalue'])
        df['test_stat_1'] = df['Test Results 1'].apply(lambda x: x['test_result']['test_stat'])
        df['h0_rejected_1'] = df['Test Results 1'].apply(lambda x: x['test_result']['h0_rejected'])

        df['alpha_2'] = df['Test Results 2'].apply(lambda x: x['test_result']['alpha'])
        df['pvalue_2'] = df['Test Results 2'].apply(lambda x: x['test_result']['pvalue'])
        df['test_stat_2'] = df['Test Results 2'].apply(lambda x: x['test_result']['test_stat'])
        df['h0_rejected_2'] = df['Test Results 2'].apply(lambda x: x['test_result']['h0_rejected'])

        df = df.drop(columns=['Test Results 1', 'Test Results 2'])
        return df 


In [50]:
import pandas as pd

class DataFrameAnalyzer:
    def __init__(self, df):
        self.df = df

    def count_nans(self):
        num_nans = self.df.isna().sum().sum()
        print(f'The DataFrame contains {num_nans} NaN values.')
        return num_nans

    def split_by_nans(self):
        mask = self.df.isna()
        nan_df = self.df[mask]
        non_nan_df = self.df[~mask]
        return nan_df, non_nan_df

    def generate_summary_table(self, non_nan_df):
        summary_table = non_nan_df.groupby(['Column Pair', 'Window']).agg({
            'pvalue_1': 'mean',
            'h0_rejected_1': 'mean',
            'pvalue_2': 'mean',
            'h0_rejected_2': 'mean'
        }).reset_index()

        summary_table.columns = [
            'Column Pair', 'Window',
            'Mean P-Value 1', 'Proportion H0 Rejected 1',
            'Mean P-Value 2', 'Proportion H0 Rejected 2'
        ]

        summary_table = summary_table.round(4)
        return summary_table



    def analyze(self):
        self.count_nans()
        nan_df, non_nan_df = self.split_by_nans()

        summary_table = self.generate_summary_table(non_nan_df)
        print('\nSummary Table:')
        print(summary_table)

        true_median_df, false_median_df = self.calculate_median_by_h0_rejected(non_nan_df)
        print('\nTrue Median DataFrame:')
        print(true_median_df)

        print('\nFalse Median DataFrame:')
        print(false_median_df)



In [82]:
def calculate_median_by_h0_rejected(df):
    # split the DataFrame into two separate DataFrames based on h0_rejected_1 value
    true_df = df[df['h0_rejected_1'] == True]
    false_df = df[df['h0_rejected_1'] == False]

    # calculate median values of pvalue_1, test_stat_1, pvalue_2, and test_stat_2 for each group
    true_median_df = true_df[['pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2']].median().to_frame().T
    true_median_df['h0_rejected_1'] = True
    true_pct = len(true_df) / len(df) * 100
    true_median_df['Percentage'] = true_pct
    true_median_df['Median Window'] = true_df['Window'].median()
    true_median_df['Median Shift'] = true_df['Shift'].median()

    false_median_df = false_df[['pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2']].median().to_frame().T
    false_median_df['h0_rejected_1'] = False
    false_pct = len(false_df) / len(df) * 100
    false_median_df['Percentage'] = false_pct
    false_median_df['Median Window'] = false_df['Window'].median()
    false_median_df['Median Shift'] = false_df['Shift'].median()

    return true_median_df, false_median_df

In [141]:
def calculate_median_by_field(df, field):
    """
    Calculates the median values of 'pvalue_1', 'test_stat_1', 'pvalue_2', and 'test_stat_2' for each group in the DataFrame,
    split by the boolean field specified.

    Args:
        df (pandas.DataFrame): The DataFrame to split and calculate median values on.
        field (str): The boolean field to split the DataFrame on.

    Returns:
        Tuple[pandas.DataFrame, pandas.DataFrame]: A tuple of two DataFrames, containing the median values for True and False groups,
        respectively. Each DataFrame also contains a 'Percentage', 'Median Window', and 'Median Shift' column.
    """
    # split the DataFrame into two separate DataFrames based on field value
    true_df = df[df[field] == True]
    false_df = df[df[field] == False]

    # calculate median values of pvalue_1, test_stat_1, pvalue_2, and test_stat_2 for each group
    true_median_df = true_df[['pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2']].median().to_frame().T
    true_median_df[field] = True
    true_pct = len(true_df) / len(df) * 100
    true_median_df['Percentage'] = true_pct
    true_median_df['Median Window'] = true_df['Window'].median()
    true_median_df['Median Shift'] = true_df['Shift'].median()

    false_median_df = false_df[['pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2']].median().to_frame().T
    false_median_df[field] = False
    false_pct = len(false_df) / len(df) * 100
    false_median_df['Percentage'] = false_pct
    false_median_df['Median Window'] = false_df['Window'].median()
    false_median_df['Median Shift'] = false_df['Shift'].median()

    return true_median_df, false_median_df


In [51]:
def fix_string_issues(s):
    s = s.replace("array(", "[")
    s = s.replace(", dtype=float64)", "]")
    s = s.replace(", 'besti'", "], 'besti'")
    s = re.sub(r"(\w+):", r'"\1":', s)  # Add quotes around keys
    s = s.replace("'", '"')  # Replace single quotes with double quotes
    s = s.replace("],]", "]]")  # Fix the remaining issue
    return s

def fix_and_parse_json(s):
    s = re.sub(r'\s+', ' ', s)  # Replace newline characters and extra spaces with single spaces
    s = re.sub(r'] ({)', r'], \1', s)  # Add missing comma between key-value pairs
    try:
        return json.loads(s)
    except json.JSONDecodeError as e:
        print(f"Error parsing: {s}")
        raise e

# remove "Window" and "Shift" from the entries and convert to float

def clean_column_values(df):
    df['Window'] = df['Window'].str.replace('window=', '').astype(int)
    df['Shift'] = df['Shift'].str.replace('shift=', '').astype(int)
    return df

def modify_row_index(df, old_row_index, new_row_index):
    df = df.rename(index={old_row_index: new_row_index})
    return df

In [55]:
def create_dataframes(dicts):
    dataframes = {}
    
    if 'volume' in dicts:
        dataframes['volume'] = pd.DataFrame.from_dict(dicts['volume'])
        
    if 'tick' in dicts:
        dataframes['tick'] = pd.DataFrame.from_dict(dicts['tick'])
        
    if 'calendar' in dicts:
        dataframes['calendar'] = pd.DataFrame.from_dict(dicts['calendar'])
        
    if 'dollar' in dicts:
        dataframes['dollar'] = pd.DataFrame.from_dict(dicts['dollar'])

    return dataframes


In [ ]:
def create_results_table(df):
    # format the columns 'pvalue_1', 'test_stat_1', 'pvalue_2', and 'test_stat_2'
    df[['pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2']] = df[['pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2']].applymap(lambda x: f'{float(x):.2e}'.replace('e-', 'x10^-') if pd.notna(x) else 'N/A')

    # replace the NaN values with 'N/A'
    df.fillna('N/A', inplace=True)

    # create the LaTeX table
    latex = df.to_latex(column_format='lcccccccc', escape=False, header=['', 'pvalue_1', 'test_stat_1', 'pvalue_2', 'test_stat_2', 'h0_rejected_1', 'Percentage', 'Median Window', 'Median Shift'], bold_rows=[0, len(df)-1], multicolumn_format='c')

    # return the LaTeX table
    return latex

In [ ]:
############################

In [52]:
symbol = 'XM1'
test =ProcessFiles(str(symbol), mmdOutputFilesLoc)

In [124]:
variable_ = test.generate_file_paths()['volume'].split("_")[-1].split(".")[0]
test.generate_file_paths()

{'volume': '/media/ak/T7/August11th2022Experiments/LinearMMDOutputFiles/XM1_volume_processedLinearMMDresults_alpha.pickle',
 'tick': '/media/ak/T7/August11th2022Experiments/LinearMMDOutputFiles/XM1_tick_processedLinearMMDresults_alpha.pickle',
 'dollar': '/media/ak/T7/August11th2022Experiments/LinearMMDOutputFiles/XM1_dollar_processedLinearMMDresults_alpha.pickle'}

In [54]:
dicts = test.read_pickles()
dicts.keys()

dict_keys(['volume', 'tick', 'dollar'])

## Get Dataframes

In [125]:
dataframes = create_dataframes(dicts)
dataframes

{'volume':     Column Pair     Window    Shift  \
 0        0 vs 2   window=5  shift=1   
 1        0 vs 2  window=10  shift=1   
 2        0 vs 2  window=15  shift=1   
 3        0 vs 2  window=20  shift=1   
 4        0 vs 2  window=25  shift=1   
 ..          ...        ...      ...   
 940    31 vs 37   window=5  shift=5   
 941    31 vs 35   window=5  shift=2   
 942    31 vs 36   window=5  shift=3   
 943    32 vs 34   window=5  shift=1   
 944    32 vs 37   window=5  shift=4   
 
                                         Test Results 1  \
 0    {'widths': [0.3127758084013007, 0.351078974559...   
 1    {'widths': [0.3127758084013007, 0.351078974559...   
 2    {'widths': [0.3127758084013007, 0.351078974559...   
 3    {'widths': [0.3127758084013007, 0.351078974559...   
 4    {'widths': [0.3127758084013007, 0.351078974559...   
 ..                                                 ...   
 940  {'widths': [0.2611623442839497, 0.293144819906...   
 941  {'widths': [0.4266564188107409

In [126]:
volumedf = DataFrameAnalyzer(test.extract_data(clean_column_values(dataframes['volume'])))
tickdf = DataFrameAnalyzer(test.extract_data(clean_column_values(dataframes['tick'])))


In [127]:
#calendardf = DataFrameAnalyzer(test.extract_data(clean_column_values(dataframes['calendar'])))
dollardf = DataFrameAnalyzer(test.extract_data(clean_column_values(dataframes['dollar'])))

In [135]:
bar_choice = 'tick'
df = tickdf

In [136]:
nandf, nonnandf =df.split_by_nans()

In [137]:
tickdf.df

,Column Pair,Window,Shift,alpha_1,pvalue_1,test_stat_1,h0_rejected_1,alpha_2,pvalue_2,test_stat_2,h0_rejected_2
0,0 vs 2,5,1,0.01,1.288445e-05,0.240125,True,0.01,2.816236e-07,0.016927,True
1,0 vs 2,10,1,0.01,1.183096e-04,0.194365,True,0.01,3.099101e-07,0.032476,True
2,0 vs 2,15,1,0.01,5.845269e-05,0.315023,True,0.01,5.345833e-06,0.025783,True
3,0 vs 2,20,1,0.01,7.770705e-04,0.065527,True,0.01,4.367276e-07,0.311985,True
4,0 vs 2,25,1,0.01,3.032374e-03,0.023825,True,0.01,7.628308e-06,0.352649,True
...,...,...,...,...,...,...,...,...,...,...,...
16620,129 vs 135,5,5,0.01,8.455850e-07,0.393999,True,0.01,4.802292e-07,0.386519,True
16621,129 vs 133,5,2,0.01,1.662210e-07,0.394086,True,0.01,6.109606e-07,0.323618,True
16622,129 vs 134,5,3,0.01,1.656596e-07,0.396689,True,0.01,NaN,0.000000,False
16623,130 vs 132,5,1,0.01,9.451097e-06,0.310168,True,0.01,1.611524e-05,0.397242,True


In [143]:
dataanalysis = DataFrameAnalyzer(nonnandf) 
_, cleandf = dataanalysis.split_by_nans()
# true, false = dataanalysis.(cleandf)
tru, fals = calculate_median_by_field(cleandf, 'h0_rejected_2')

In [148]:
pd.concat([tru, fals], axis=0)

,pvalue_1,test_stat_1,pvalue_2,test_stat_2,h0_rejected_2,Percentage,Median Window,Median Shift
0,0.000001,0.175688,0.000001,0.174784,True,98.544361,40.0,5.0
0,0.000002,0.191485,0.012767,0.000000,False,1.455639,10.0,5.0


In [132]:


# display the formatted dataframe
filename = "_".join((str(symbol), str(bar_choice), str(variable_), 'summary_table.csv'))
combined_filepath = os.path.join(tables, filename )
combined_df = pd.concat([calculate_median_by_h0_rejected(nonnandf)[0], calculate_median_by_h0_rejected(nonnandf)[1]], axis=0)

In [133]:
df_formatted

,pvalue_1,test_stat_1,pvalue_2,test_stat_2,h0_rejected_1,Percentage,Median Window,Median Shift
0,1.82x10^-05,1.01x10^-01,0.000008,0.159963,True,23.703704,15.0,2.0
0,nan,nan,0.000002,0.091767,False,76.296296,10.0,5.0
tick_0,1.44x10^-06,1.76x10^-01,0.000001,0.173000,True,100.000000,40.0,5.0
tick_1,nan,nan,NaN,NaN,False,0.000000,NaN,NaN
dollar_0,5.47x10^-06,3.17x10^-02,0.000005,0.032700,True,27.860000,10.0,3.0
dollar_1,nan,nan,NaN,0.000000,False,72.140000,10.0,5.0
volume_0,1.82x10^-05,1.01x10^-01,0.000003,0.125000,True,43.170000,10.0,3.0
volume_1,nan,nan,NaN,0.000000,False,56.830000,10.0,5.0


In [97]:
df_formatted.to_csv(combined_filepath )

,pvalue_1,test_stat_1,pvalue_2,test_stat_2,h0_rejected_1,Percentage,Median Window,Median Shift
0,0.000018,0.100958,0.000008,0.159963,True,23.703704,15.0,2.0
0,NaN,NaN,0.000002,0.091767,False,76.296296,10.0,5.0


In [72]:

# 
# filepath = os.path.join(tables, filename )
# # calculate_median_by_h0_rejected(cleandf).to_csv(filepath)

In [68]:
# print(filepath)
# pd.read_csv(filepath,index_col=0)

In [100]:
os.listdir(tables)

['G_1_tick_alpha_summary_table.csv',
 'G_1_dollar_alpha_summary_table.csv',
 'G_1_volume_alpha_summary_table.csv',
 'G_1_calendar_alpha_summary_table.csv',
 'XM1_volume_alpha_summary_table.csv']

In [73]:
# pd.read_csv('/media/ak/T7/August11th2022Experiments/TablesOfResults/G_1_volume_alpha_summary_table.csv',index_col=0)

In [74]:
# pd.read_csv('/media/ak/T7/August11th2022Experiments/TablesOfResults/G_1_tick_alpha_summary_table.csv',index_col=0)

In [75]:
# pd.read_csv('/media/ak/T7/August11th2022Experiments/TablesOfResults/G_1_dollar_alpha_summary_table.csv',index_col=0)

In [109]:

def load_and_combine_tables(tables_path, symbol_):
    # create empty dataframe to hold the combined data
    combined_df = pd.DataFrame()
   

    # loop through each file in the directory
    for filename in files:
        if filename.endswith('.csv'):
            # extract the alpha type from the filename
            alpha_type = filename.split('_')[2]

            # load the csv file into a dataframe
            df = pd.read_csv(os.path.join(tables_path, filename), index_col=0)

            # modify the index labels to include the alpha type
            df.index = [f'{alpha_type}_0', f'{alpha_type}_1']

            # append the dataframe to the combined dataframe
            combined_df = pd.concat([combined_df, df], axis=0)

    return combined_df

In [ ]:

# format the 'pvalue_1' and 'test_stat_1' columns
df_formatted = format_columns(combined_df, ['pvalue_1', 'test_stat_1'])

In [123]:
# symbol_ ='G_1'
# files = [f for f in os.listdir(tables_path) if str(symbol_) in f]
# for filename in files:
#     alpha_type = filename.split('_')[2]
#     # load the csv file into a dataframe
#     df = pd.read_csv(os.path.join(tables_path, filename), index_col=0)
#     print('#####################',filename)
#     print(df)
# #     # modify the index labels to include the alpha type
# #     df.index = [f'{alpha_type}_0', f'{alpha_type}_1']
# #     # append the dataframe to the combined dataframe
# #     combined_df = pd.concat([combined_df, df], axis=0)

In [121]:
combined_all_infos_df = pd.read_csv(os.path.join(tables_path,'G_1_calendar_alpha_summary_table.csv'), index_col=0)

In [122]:
print(combined_all_infos_df.to_latex())

\begin{tabular}{lllrrlrrr}
\toprule
{} &     pvalue\_1 &  test\_stat\_1 &  pvalue\_2 &  test\_stat\_2 &  h0\_rejected\_1 &  Percentage &  Median Window &  Median Shift \\
\midrule
tick\_0     &  1.44x10\textasciicircum -06 &  1.76x10\textasciicircum -01 &  0.000001 &       0.1730 &           True &      100.00 &           40.0 &           5.0 \\
tick\_1     &          NaN &          NaN &       NaN &          NaN &          False &        0.00 &            NaN &           NaN \\
dollar\_0   &  5.47x10\textasciicircum -06 &  3.17x10\textasciicircum -02 &  0.000005 &       0.0327 &           True &       27.86 &           10.0 &           3.0 \\
dollar\_1   &          NaN &          NaN &       NaN &       0.0000 &          False &       72.14 &           10.0 &           5.0 \\
volume\_0   &  1.82x10\textasciicircum -05 &  1.01x10\textasciicircum -01 &  0.000003 &       0.1250 &           True &       43.17 &           10.0 &           3.0 \\
volume\_1   &          NaN &          NaN & 